# Notebook 2: Application d'analyse immobilière
---
## Table des matières

1. [Présentation du projet](#Présentation-du-projet)
2. [Objectifs de l'application](#Objectifs-de-l'application)
3. [Sources de données](#Sources-de-données)
4. [Import des bibliothèques](#Import-des-bibliothèques)
5. [Chargement des données](#Chargement-des-données)
6. [Vue d'ensemble du marché](#Vue-d'ensemble-du-marché)
7. [Exploration interactive par budget](#Exploration-interactive-par-budget)
8. [Analyse géographique](#Analyse-géographique)
9. [Système de recommandation](#Système-de-recommandation)
10. [Analyse prédictive](#Analyse-prédictive)
11. [Tableaux de bord personnalisés](#Tableaux-de-bord-personnalisés)
12. [Conclusions et recommandations](#Conclusions-et-recommandations)
13. [Annexes](#Annexes)
14. [Guide d'utilisation](#Guide-d'utilisation)

---

## Présentation du projet
### Persona ciblé

#### Profil
**Nom**: Marie
**Age**: 40 ans
**Occupation**: cadre supérieur à Paris

**Budget**: 300-500k€

**Objectif**: Acheter son premier bien locatif pour générer des revenus passifs


#### Besoins spécifiques
1. Identifier les villes avec le meilleur rendement locatif dans IDF
2. Comprendre quels types de biens sont les plus rentables (studio vs T3 pour coloc)
3. Calculer la rentabilité brute et nette
4. Comparer le potentiel de location simple vs coloc
5. Analyser les proximité des transport en commun
6. Trouver des biens accessibles avec son budget
7. Avoir un aperçu clair du marché (prix au m2,tendances)
8. Observer l’évolution des prix sur 5–10 ans pour anticiper une plus-value
9. Évaluer la liquidité du marché
10. Obtenir un classement des Top 10 opportunités actuelles (un tableau ou score automatique).
11. Analyser les charges moyennes par type de logement
12. Vérifier la sécurité et l’attractivité du quartier


---

## Objectifs de l'application
<!-- COMPLÉTEZ ICI: Listez les objectifs principaux de votre application -->
<!-- Exemple: -->
<!-- - Identifier les zones à fort potentiel de rentabilité -->
<!-- - Calculer le ROI d'un investissement immobilier -->
<!-- - etc. -->
Identifier des territoires prometteurs pour un premier investissement locatif

---

## Sources de données
<!-- COMPLÉTEZ ICI: Décrivez vos sources de données -->
<!-- - Données principales (DVF, etc.) -->
<!-- - Données complémentaires (INSEE, API, etc.) -->
<!-- - Période couverte -->
<!-- - Nombre de transactions analysées -->

---

## Import des bibliothèques

In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import seaborn as sns

# Pour les visualisations interactives
import plotly.express as px
import plotly.graph_objects as go 
import ipywidgets as widgets
import matplotlib.pyplot as plt

import folium
import geopandas as gpd
from folium.plugins import MarkerCluster

# CODEZ ICI: Configuration supplémentaire si nécessaire
import branca.colormap as cm

---

## Chargement des données

In [2]:
# --- Chargement des données nettoyées (préparées dans le Notebook 1) ---
# DVF
dvf_df = pd.read_csv('data/raw/full_2024.csv')

# Communes
communes_df = pd.read_csv('data/cleaned/communes.csv')

# Indicateurs de loyer
loyers_df = pd.read_csv('data/cleaned/loyers_IDF_2024.csv')  ## already with latitude and longitude data

# Transports
transports_gdf = gpd.read_file("data/transports_nettoyes.geojson")



C:\Users\zahor\AppData\Local\Temp\ipykernel_2980\3393862134.py:3: DtypeWarning: Columns (8,10,12,14,16,17,18,20,22,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  dvf_df = pd.read_csv('data/raw/full_2024.csv')


---

## **Vue d'ensemble du marché**

### **Widget 1.1 : Rendements locatifs en IDF**
--> identify the cities with the best local values
- best average selling price
- best average renting price


In [3]:
# clean dvf a bit
def preclean_clean(dvf_df):
    # Filter to IDF departments
    dvf_df = dvf_df[dvf_df["code_departement"].isin([75, 77, 78, 91, 92, 93, 94, 95])]

    # Filter to only apartments and houes
    dvf_df = dvf_df[dvf_df["type_local"].isin(["Appartement", "Maison"])]

    return dvf_df

def categorise_type_bien(row):
    if row['type_local'] == 'Maison':
        return 'Maison'
    elif row['type_local'] == 'Appartement':
        if pd.isna(row['nombre_pieces_principales']) or row['nombre_pieces_principales'] == 0:
            return 'Appartement'
        elif row['nombre_pieces_principales'] <= 2:
            return 'Appartement T1-T2'
        elif row['nombre_pieces_principales'] >= 3:
            return 'Appartement T3+'
    return row['type_local']  # Fallback to original type if no condition matches

def create_dvf_rent_dataset(dvf_df=dvf_df, loyers_df=loyers_df):
    # Clean a bit
    try:
        dvf_df = preclean_clean(dvf_df)
    except:
        pass
    dvf_df['type_local'] = dvf_df.apply(categorise_type_bien, axis=1)

    
    # Agréger les données DVF par commune
    df_prix = dvf_df.groupby(['code_commune', 'type_local']).agg({
        'valeur_fonciere': 'mean',
        'surface_reelle_bati': 'mean',
        'id_mutation': 'count'
    }).reset_index()

    df_prix['prix_m2'] = df_prix['valeur_fonciere'] / df_prix['surface_reelle_bati']

    # Merge with rents
    df_complet = df_prix.merge(
        loyers_df,
        left_on=['code_commune', 'type_local'],
        right_on=['INSEE_C', 'type_bien'])
    df_complet = df_complet.drop(columns=['INSEE_C'])

    # Calculer le rendement brut
    df_complet['loyer_annuel'] = df_complet['loyer_predit_m2'] * df_complet['surface_reelle_bati'] * 12
    df_complet['rendement_brut'] = (df_complet['loyer_annuel'] / df_complet['valeur_fonciere']) * 100

    return df_complet

df_dvf_rent = create_dvf_rent_dataset()

In [4]:
def create_rental_yield_map(df_complet, min_yield=None, max_yield=None):
    df_complet = df_complet.copy()
    
    # Handle potential NaN or infinite values
    df_complet = df_complet.dropna(subset=['rendement_brut', 'latitude', 'longitude'])
    
    # Create a base map centered on Paris
    m = folium.Map(location=[48.8566, 2.3522], zoom_start=9)
    
    # Color mapping
    # Use median and IQR for more robust scaling
    q1 = df_complet['rendement_brut'].quantile(0.25)
    q3 = df_complet['rendement_brut'].quantile(0.75)
    iqr = q3 - q1
    
    # Define color range
    color_min = max(df_complet['rendement_brut'].min(), q1 - 1.5 * iqr)
    color_max = min(df_complet['rendement_brut'].max(), q3 + 1.5 * iqr)
    
    # Ensure color_min and color_max are different
    if color_min == color_max:
        color_min = df_complet['rendement_brut'].min()
        color_max = df_complet['rendement_brut'].max()
    
    # Create color map
    color_map = cm.LinearColormap(
        colors=['red', 'yellow', 'green'],
        vmin=color_min,
        vmax=color_max
    )
    
    # Add color map to the map
    color_map.add_to(m)
    color_map.caption = f'Rendement Brut (%) [Visualized Range: {color_min:.2f} - {color_max:.2f}]'
    
    # Create a marker cluster group
    marker_cluster = MarkerCluster().add_to(m)
    
    # Custom color function to handle outliers
    def get_marker_color(value):
        if color_min <= value <= color_max:
            return color_map(value)
        elif value > color_max:  # above range, use max color (green)
            return 'green'
        else:  # below range, use min color (red)
            return 'red'
    
    # Add markers for each commune in filtered dataframe
    for idx, row in df_complet.iterrows():
        try:
            # Determine marker color based on special outlier handling
            color = get_marker_color(row['rendement_brut'])
            
            # Calculate marker size based on transaction volume (scaled down)
            marker_size = max(5, min(20, row['id_mutation'] / 10))
            
            # Create popup content with actual value and outlier note if applicable
            popup_content = f"""
            <b>{row.get('nom_commune', 'Commune non identifiée')} ({row['type_bien']})</b><br>
            Prix/m²: {row['prix_m2']:.0f} €<br>
            Loyer/m²: {row['loyer_predit_m2']:.2f} €<br>
            <strong>Rendement Brut: {row['rendement_brut']:.2f}%</strong><br>
            Transactions (vente): {row['id_mutation']}
            Transactions (location): {row['nombre_observations_commune']}
            {'<p style="color:red;">NOTE: Exceptionally high yield</p>' if row['rendement_brut'] > color_max else ''}
            """
            
            # Add marker
            folium.CircleMarker(
                location=[row['latitude'], row['longitude']],
                radius=marker_size,
                popup=popup_content,
                color=color,
                fill=True,
                fillColor=color,
                fillOpacity=0.7
            ).add_to(marker_cluster)
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
    
    return m

def create_interactive_map(df_complet):
    # Compute initial min and max
    min_possible = df_complet['rendement_brut'].min()
    max_possible = df_complet['rendement_brut'].max()
    
    # Create output widget to display map
    output = widgets.Output()
    
    # Create slider widgets
    min_slider = widgets.FloatSlider(
        value=min_possible,
        min=min_possible,
        max=max_possible,
        step=(max_possible - min_possible) / 100,
        description='Min Yield:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.2f'
    )
    
    max_slider = widgets.FloatSlider(
        value=max_possible,
        min=min_possible,
        max=max_possible,
        step=(max_possible - min_possible) / 100,
        description='Max Yield:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.2f'
    )
    
    # Function to update map
    def update_map(change):
        # Clear previous output
        with output:
            clear_output(wait=True)
            
            # Create new map with current slider values
            min_val = min_slider.value
            max_val = max_slider.value
            
            # Ensure min is less than max
            if min_val > max_val:
                min_val, max_val = max_val, min_val
            
            # Create and display map
            m = create_rental_yield_map(df_complet, min_val, max_val)
            display(m)
    
    # Initial map rendering
    with output:
        m = create_rental_yield_map(df_complet)
        display(m)
    
    # Link sliders to update function
    min_slider.observe(update_map, names='value')
    max_slider.observe(update_map, names='value')
    
    # Arrange widgets
    return widgets.VBox([
        widgets.HBox([min_slider, max_slider]),
        output
    ])

# # Outliers
# outliers = df_dvf_rent[df_dvf_rent['rendement_brut'] > 100]
# print("Outliers with rendement brut > 100%:")
# print(outliers[['nom_commune', 'rendement_brut']])

# Create and display interactive map
interactive_map = create_interactive_map(df_dvf_rent)
display(interactive_map)

### **Widget 1.2 Comparateur de types de biens**
Répond au besoin : Comprendre quels types de biens sont les plus rentable
Fonction : tableau interactif qui compare rentabilité par type de bien dans une ville donnée.

In [5]:
dvf_df_clean = preclean_clean(dvf_df)

dvf_df_clean['type_local'] = dvf_df_clean.apply(categorise_type_bien, axis=1)

print(dvf_df_clean.columns)
commune_type_counts = (
    dvf_df_clean.groupby(['code_departement', 'code_commune', 'nom_commune', 'type_local'])
      .size()
      .reset_index(name='nb_occurrences')
)

def convert_to_categorial(dvf_df_clean, categorial_columns):
    """Converts specified columns to categorial type"""
    converted = []
    for col in categorial_columns:
        if col not in df.columns:
            print(f"⚠ Colonne '{col}' non trouvée, ignorer")
            continue
        
        try:
            df[col] = df[col].astype(str).str.strip()
            df[col] = df[col].astype('object')
            converted.append(col)
        except Exception as e:
            print(f"⚠ Erreur de conversion de '{col}' : {e}")
    
    print(f"✓ Converti {len(converted)} colonnes en type catégoriel")
    return df

count_df = convert_to_categorial(commune_type_counts, ['code_commune'])

dept_options = sorted(count_df['code_departement'].unique())
dept_dropdown = widgets.Dropdown(
    options=dept_options,
    description='Département:',
    value=dept_options[0]
)

# Define the update function
def update_plot(selected_dept):
    filtered_df = count_df[count_df['code_departement'] == selected_dept]
    fig = px.bar(
        filtered_df,
        x='code_commune',
        y='nb_occurrences',
        color='type_local',
        title=f'Nombre des biens par type et commune (Département {selected_dept})',
        labels={
            'code_commune': 'Code commune',
            'nom_commune': 'Commune',
            'nb_occurrences': 'Nombre de biens',
            'type_local': 'Type de bien'
        },
        barmode='stack',
        custom_data=['type_local', 'code_commune', 'nom_commune'],
        width=900,
        height=600
    )
    fig.update_traces(
        hovertemplate=
            "Nombre de bien: %{y} (%{customdata[0]})<br>" +
            "Commune: %{customdata[1]} (%{customdata[2]})<extra></extra>"
    )
    fig.show()

# Interactive widget connection
# widgets.interact(update_plot, selected_dept=dept_dropdown)


Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'ancien_code_commune',
       'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')


NameError: name 'df' is not defined

In [ ]:
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display

def compare_property_types(df):
    # Group by commune and property type, calculating mean rental yield
    grouped_df = df.groupby(['nom_commune', 'type_local'])['rendement_brut'].agg([
        'mean',  # Average rental yield
        'count'  # Number of properties
    ]).reset_index()

    display(grouped_df)
    
    # Compute commune-level coordinates (mean of all properties in the commune)
    commune_coords = df.groupby('nom_commune')[['latitude', 'longitude']].mean()
    
    # Create dropdown for commune selection
    commune_dropdown = widgets.Dropdown(
        options=sorted(df['nom_commune'].unique()),
        description='Commune:',
        disabled=False
    )
    
    # Output widget to display map
    map_output = widgets.Output()
    
    def update_map(change):
        # Clear previous map
        map_output.clear_output(wait=True)
        
        # Filter for selected commune
        selected_commune = commune_dropdown.value
        commune_data = grouped_df[grouped_df['nom_commune'] == selected_commune]
        
        # Create map for the selected commune
        with map_output:
            # Get coordinates for the selected commune
            try:
                center_lat = commune_coords.loc[selected_commune, 'latitude']
                center_lon = commune_coords.loc[selected_commune, 'longitude']
                m = folium.Map(location=[center_lat, center_lon], zoom_start=12)
            except Exception as e:
                # Fallback to a default location if coordinates are not available
                print(f"Error getting coordinates: {e}")
                m = folium.Map(location=[48.8566, 2.3522], zoom_start=10)  # Paris as default
            
            # Print the grouped data for the selected commune
            print(f"\nRental Yield Comparison for {selected_commune}:")
            print(commune_data.to_string(index=False))
            
            # Color palette for different property types
            color_map = {
                'Appartement': 'blue',
                'Maison': 'green',
                'Studio': 'red',
                'Terrain': 'purple'
            }
            
            # Add markers to the map for each property type
            for _, row in commune_data.iterrows():
                # Use a default color if property type not in color map
                marker_color = color_map.get(row['type_local'], 'gray')
                
                # Create a more informative popup
                popup_content = f"{row['type_local']}\nAvg Yield: {row['mean']:.2f}%\nCount: {row['count']:,}"
                
                folium.Marker(
                    location=[center_lat, center_lon],
                    popup=popup_content,
                    tooltip=row['type_local'],
                    icon=folium.Icon(color=marker_color, icon='home')
                ).add_to(m)
            
            # Save the map as an HTML file
            m.save(f'{selected_commune}_property_map.html')
            
            # Display the map
            display(m)
    
    # Initial update
    commune_dropdown.observe(update_map, names='value')
    update_map(None)
    
    # Combine widgets
    return widgets.VBox([
        commune_dropdown,
        map_output
    ])

# Create the widget
property_type_comparison = compare_property_types(df_dvf_rent)
display(property_type_comparison)

,nom_commune,type_local,mean,count
0,Ableiges,Appartement T3+,6.665093,1
1,Ableiges,Maison,4.907457,1
2,Ablis,Appartement T1-T2,7.053292,1
3,Ablis,Appartement T3+,6.372718,1
4,Ablis,Maison,5.049579,1
...,...,...,...,...
2294,Évry-Grégy-sur-Yerre,Appartement T1-T2,13.214647,1
2295,Évry-Grégy-sur-Yerre,Maison,5.600926,1
2296,Ézanville,Appartement T1-T2,7.092470,1
2297,Ézanville,Appartement T3+,7.626328,1


### Widget 1.3
<!-- COMPLÉTEZ ICI: Décrivez le type de graphique et les insights attendus -->


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output

def preprocess_dvf_data(df):
    """
    Preprocess the DVF dataset
    
    Args:
        df (pandas.DataFrame): Raw DVF dataset
    
    Returns:
        pandas.DataFrame: Preprocessed dataset
    """
    # Calculate price per m²
    df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']
    
    # Categorize property types
    def categorize_property(row):
        if row['type_local'] in ['Appartement']:
            return 'Appartement'
        elif row['type_local'] == 'Appartement T1-T2':
            return 'Appartements T2-T3'
        elif row['type_local'] == 'Appartement T3+':
            return 'Appartement T3+'
        elif row['type_local'] == "Maison":
        else:
            return 'Autres'
    
    df['category_bien'] = df.apply(categorize_property, axis=1)
    
    return df

def analyze_property_types(df, commune, selected_categories=None):
    """
    Perform comprehensive analysis of property types in a given commune
    
    Args:
        df (pandas.DataFrame): Preprocessed DVF dataset
        commune (str): Name of the commune to analyze
        selected_categories (list, optional): Categories to filter
    
    Returns:
        dict: Analysis results and visualizations
    """
    # Filter data for specific commune
    df_commune = df[df['nom_commune'] == commune].copy()
    
    # Filter by selected categories if provided
    if selected_categories:
        df_commune = df_commune[df_commune['category_bien'].isin(selected_categories)]
    
    # 1. Boxplot: Prix au m² par catégorie de bien
    fig_prix_m2 = px.box(
        df_commune, 
        x='category_bien', 
        y='prix_m2',
        title=f'Prix au m² par catégorie de bien à {commune}',
        labels={
            'prix_m2': 'Prix au m²', 
            'category_bien': 'Catégorie de bien'
        }
    )
    
    # 2. Répartition des types de biens
    property_dist = df_commune['category_bien'].value_counts()
    fig_distribution = px.pie(
        values=property_dist.values, 
        names=property_dist.index,
        title=f'Répartition des types de biens à {commune}'
    )
    
    # 3. Tableau récapitulatif des métriques
    metriques_resumees = df_commune.groupby('category_bien').agg({
        'valeur_fonciere': ['count', 'mean'],
        'prix_m2': ['mean', 'median'],
        'surface_reelle_bati': ['mean', 'median']
    }).round(2)
    
    metriques_resumees.columns = [
        'Nombre de biens', 
        'Prix moyen', 
        'Prix/m² moyen', 
        'Prix/m² médian', 
        'Surface moyenne', 
        'Surface médiane'
    ]
    
    return {
        'boxplot_prix_m2': fig_prix_m2,
        'distribution_biens': fig_distribution,
        'metriques_resumees': metriques_resumees
    }

def create_interactive_dvf_comparator(df):
    """
    Create an interactive DVF property type comparator using ipywidgets
    
    Args:
        filepath (str): Path to the DVF CSV file
    """
    # Load and preprocess data
    df_preprocessed = preprocess_dvf_data(df)
    
    # Get unique communes and categories
    communes = sorted(df_preprocessed['nom_commune'].unique())
    categories = sorted(df_preprocessed['category_bien'].unique())
    
    # Create widgets
    commune_dropdown = widgets.Dropdown(
        options=communes,
        description='Commune:',
        disabled=False
    )
    
    category_multiselect = widgets.SelectMultiple(
        options=categories,
        description='Categories:',
        disabled=False,
        layout=widgets.Layout(width='300px')
    )
    
    output = widgets.Output()
    
    def on_update(change):
        # Clear previous output
        with output:
            clear_output(wait=True)
            
            # Get selected commune and categories
            commune = commune_dropdown.value
            selected_categories = category_multiselect.value or categories
            
            # Perform analysis
            try:
                resultats = analyze_property_types(
                    df_preprocessed, 
                    commune, 
                    selected_categories
                )
                
                # Display metrics
                print(f"Analyse comparative des types de biens à {commune}")
                print("\n--- Métriques Résumées ---")
                display(resultats['metriques_resumees'])
                
                # Display visualizations
                print("\n--- Visualisations ---")
                resultats['boxplot_prix_m2'].show()
                resultats['distribution_biens'].show()
            
            except Exception as e:
                print(f"Erreur lors de l'analyse : {e}")
    
    # Link widgets to update function
    commune_dropdown.observe(on_update, names='value')
    category_multiselect.observe(on_update, names='value')
    
    # Layout
    form = widgets.VBox([
        widgets.HBox([commune_dropdown, category_multiselect]),
        output
    ])
    
    # Display the interactive form
    display(form)
    
    # Trigger initial update
    with output:
        clear_output(wait=True)
        # Use first commune by default
        resultats = analyze_property_types(
            df_preprocessed, 
            communes[0]
        )
        print(f"Analyse comparative des types de biens à {communes[0]}")
        print("\n--- Métriques Résumées ---")
        display(resultats['metriques_resumees'])
        print("\n--- Visualisations ---")
        resultats['boxplot_prix_m2'].show()
        resultats['distribution_biens'].show()

# Usage example (to be run in Jupyter Notebook):
# create_interactive_dvf_comparator('path/to/your/dvf_dataset.csv')

# Exemple d'utilisation
create_interactive_dvf_comparator(df_dvf_rent)

## Exploration interactive par budget

### Widget 2.1 : Explorateur de budget (interactif - 1 à 3 paramètres)
<!-- COMPLÉTEZ ICI: Décrivez les paramètres interactifs -->
<!-- Exemple: -->
<!-- - Slider pour le budget (min-max) -->
<!-- - Menu déroulant pour le type de bien -->
<!-- - Slider pour la surface minimale -->

In [ ]:
# CODEZ ICI: Créer les widgets interactifs (sliders, dropdowns, etc.)

# CODEZ ICI: Fonction de filtrage basée sur les paramètres

# CODEZ ICI: Affichage des résultats (tableau + graphique)

### Widget 2.2 : Calculateur de rentabilité locative (interactif - 1 à 3 paramètres)
<!-- COMPLÉTEZ ICI: Décrivez les paramètres -->
<!-- Exemple: Prix d'achat, loyer mensuel estimé, charges, etc. -->


In [ ]:
# CODEZ ICI: Créer les inputs interactifs

# CODEZ ICI: Fonction de calcul du ROI, rendement brut, rendement net

# CODEZ ICI: Affichage des résultats avec visualisation

---

## Analyse géographique

### Widget 3.1 : Comparaison régionale (interactif - 1 à 3 paramètres)

<!-- COMPLÉTEZ ICI: Décrivez la comparaison attendue -->
<!-- Exemple: Comparer 2-3 régions/villes sur plusieurs critères -->


In [ ]:
# CODEZ ICI: Sélecteur de régions/villes à comparer

# CODEZ ICI: Calcul des métriques de comparaison

# CODEZ ICI: Visualisation comparative (barres, radar chart, etc.)

### Widget 3.2 : Carte interactive des prix (complexe - carte + paramètres)

<!-- COMPLÉTEZ ICI: Décrivez les fonctionnalités de la carte -->
<!-- Exemple: -->
<!-- - Carte choroplèthe des prix moyens -->
<!-- - Filtres par type de bien -->
<!-- - Pop-up avec détails au clic -->
<!-- - Layers pour différentes métriques -->

In [ ]:
# CODEZ ICI: Créer la carte interactive avec Folium ou Plotly

# CODEZ ICI: Ajouter les paramètres de filtrage

# CODEZ ICI: Afficher la carte


### Widget 3.3 : Carte interactive (transports et prix moyens)

In [ ]:
def carte_transport(transports_gdf):
    # --- Création de la carte ---
    m = folium.Map(
        location=[48.8566, 2.3522],  # centre de Paris
        zoom_start=9,
        tiles="CartoDB positron"
    )

    # --- Couleurs par type de transport (pour organisation) ---
    couleurs = {
        "METRO": "#800080",   # violet
        "RER": "#E41A1C",     # rouge
        "TRAIN": "#377EB8",   # bleu
        "TRAM": "#4DAF4A",    # vert
    }

    # --- Création d'un cluster par type ---
    for t in couleurs.keys():
        subset = transports_gdf[transports_gdf["type"] == t]
        cluster_type = MarkerCluster(name=f"{t}").add_to(m)

        for _, row in subset.iterrows():
            picto = row["picto_url"]

            popup_html = f"""
            <div style='font-size:13px; line-height:1.4'>
                <b>{row['nom']}</b><br>
                🚈 Type : {t}<br>
                🧭 Ligne : {row['ligne']}<br>
                🏢 Exploitant : {row['exploitant']}<br>
                <img src="{picto}" width="50"><br>
                <i>Coordonnées : {round(row['longitude'], 3)}, {round(row['latitude'], 3)}</i>
            </div>
            """

            try:
                icon = folium.CustomIcon(
                    icon_image=picto,
                    icon_size=(30, 30),
                    icon_anchor=(15, 15)
                )

                folium.Marker(
                    location=[row.geometry.y, row.geometry.x],
                    popup=folium.Popup(popup_html, max_width=300),
                    icon=icon
                ).add_to(cluster_type)

            except Exception as e:
                print(f"⚠️ Erreur sur {row['nom']}: {e}")

    # --- Contrôle des couches ---
    folium.LayerControl(collapsed=False).add_to(m)

    # --- Sauvegarde ---
    m.save("data/carte_transports_avec_icones.html")

    return m

carte_transport(transports_gdf)


---

## Système de recommandation

### Widget 4.1 : Détecteur d'opportunités (complexe - plus de 3 paramètres)

<!-- COMPLÉTEZ ICI: Décrivez tous les paramètres du système de recommandation -->
<!-- Exemple: -->
<!-- - Budget min/max -->
<!-- - Type(s) de bien souhaité(s) -->
<!-- - Rendement minimum attendu -->
<!-- - Région(s) préférée(s) -->
<!-- - Niveau de risque acceptable -->
<!-- - etc. -->

In [ ]:
# CODEZ ICI: Créer tous les widgets de paramétrage

# CODEZ ICI: Fonction de scoring/recommandation

# CODEZ ICI: Affichage des Top N opportunités recommandées

# CODEZ ICI: Visualisation des recommandations (tableau + carte/graphique)

---

## Tableaux de bord personnalisés

### Widget 6.1 : Dashboard personnalisé

<!-- COMPLÉTEZ ICI: Décrivez les éléments du dashboard -->
<!-- Un dashboard qui combine plusieurs métriques importantes pour le persona -->


In [ ]:
# CODEZ ICI: Créer un dashboard multi-graphiques

# CODEZ ICI: Possibilité de personnaliser les métriques affichées

---

## Conclusions et recommandations

### Synthèse des analyses
<!-- COMPLÉTEZ ICI: Rédigez les conclusions principales -->
<!-- Basées sur les analyses effectuées dans les sections précédentes -->

### Recommandations pour l'investisseur
<!-- COMPLÉTEZ ICI: Listez les recommandations concrètes -->
<!-- Exemple: -->
<!-- 1. Les 3 meilleures zones identifiées -->
<!-- 2. Le type de bien optimal -->
<!-- 3. Les zones à éviter -->
<!-- 4. Conseils stratégiques -->

### Points d'attention
<!-- COMPLÉTEZ ICI: Listez les limites et précautions -->
<!-- Exemple: Limites des données, facteurs non pris en compte, etc. -->

---

## Annexes

### Méthodologie
<!-- COMPLÉTEZ ICI: Expliquez votre méthodologie -->
<!-- - Comment les données ont été nettoyées -->
<!-- - Formules de calcul utilisées -->
<!-- - Choix techniques justifiés -->

### Sources et références
<!-- COMPLÉTEZ ICI: Listez toutes vos sources -->
<!-- - Liens vers les datasets -->
<!-- - Documentation des APIs utilisées -->
<!-- - Références bibliographiques si applicable -->

---

## Guide d'utilisation

### Comment utiliser ce notebook
<!-- COMPLÉTEZ ICI: Instructions pour l'utilisateur final -->
<!-- 1. Exécuter les cellules dans l'ordre -->
<!-- 2. Comment interagir avec les widgets -->
<!-- 3. Comment interpréter les résultats -->
<!-- etc. -->

### Dépendances requises

In [ ]:
# Liste des bibliothèques nécessaires
# COMPLÉTEZ ICI: Listez toutes les bibliothèques et leurs versions
# pandas==X.X.X
# matplotlib==X.X.X
# etc.

---

**Projet développé par :**
- Ashley OHNONA
- Harisoa RANDRIANASOLO
- Fairouz YOUDARENE
- Jennifer ZAHORA

**Date :** <!-- COMPLÉTEZ ICI: Date -->

**Version :** 1.0